<a href="https://colab.research.google.com/github/aadhamashraf/Real-Time-Road-Anomaly-Detection-for-Autonomous-Vehicles/blob/main/modeling/RDD_2022_Road_Damage_DetectionVGG16%2619.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

aliabdelmenam_rdd_2022_path = kagglehub.dataset_download('aliabdelmenam/rdd-2022')
ziadalsawy_rdd_2022_path = kagglehub.dataset_download('ziadalsawy/rdd-2022')

print('Data source import complete.')


# **Handling YOLO Formats and Generating Vectors**

## **How many Unique Class we Have ?**

In [ ]:
# import os

# def count_yolo_classes(label_dir):
#     class_ids = set()

#     for filename in os.listdir(label_dir):
#         if filename.endswith(".txt"):
#             filepath = os.path.join(label_dir, filename)
#             with open(filepath, 'r') as f:
#                 for line in f:
#                     parts = line.strip().split()
#                     if parts:  # avoid empty lines
#                         class_id = int(parts[0])
#                         class_ids.add(class_id)

#     print(f"Number of unique classes: {len(class_ids)}")
#     print(f"Classes found: {sorted(class_ids)}")

# directory_path = "/kaggle/input/rdd-2022/RDD_SPLIT/train/labels"
# count_yolo_classes(directory_path)

KeyboardInterrupt: 

## **Frequency Resolving for each Label**

In [ ]:
# import os
# from collections import Counter

# labels_dir = '/kaggle/input/rdd-2022/RDD_SPLIT/train/labels'
# all_classes = []

# for label_file in os.listdir(labels_dir):
#     if not label_file.endswith('.txt'):
#         continue
#     with open(os.path.join(labels_dir, label_file), 'r') as f:
#         for line in f:
#             class_id = int(line.strip().split()[0])
#             all_classes.append(class_id)

# print(Counter(all_classes))

Counter({0: 18201, 1: 8386, 3: 7554, 2: 7527, 4: 4628})


## **Generating the CSV Files**

In [ ]:
# import os
# import shutil
# import pandas as pd

# # Define class mappings (human-readable damage labels)
# class_id_to_name = {
#     0: 'longitudinal_crack',
#     1: 'transverse_crack',
#     2: 'alligator_crack',
#     3: 'pothole',
#     4: 'other_damage'
# }

# class_names = list(class_id_to_name.values())
# num_classes = len(class_names)

# # Dataset paths
# original_base = '/kaggle/input/rdd-2022/RDD_SPLIT'
# splits = ['train', 'val', 'test']
# new_base = '/kaggle/working/'

# # Loop through the splits (train, val, test) to process CSVs
# for split in splits:
#     labels_dir = os.path.join(original_base, split, 'labels')
#     data = []

#     # Process each label file in the 'labels' directory
#     for label_file in os.listdir(labels_dir):
#         if not label_file.lower().endswith('.txt'):
#             continue

#         label_path = os.path.join(labels_dir, label_file)
#         label_vector = [0] * num_classes

#         with open(label_path, 'r') as f:
#             for line in f:
#                 class_id = int(line.strip().split()[0])
#                 if class_id in class_id_to_name:
#                     label_vector[class_id] = 1

#         # Assuming you have a naming convention for image files (e.g., the label file name matches the image file name)
#         img_file = label_file.replace('.txt', '.jpg')  # or .png, .jpeg based on your dataset format
#         data.append([img_file] + label_vector)

#     # Save the CSV file for the current split
#     df = pd.DataFrame(data, columns=['filename'] + class_names)
#     df.to_csv(os.path.join(new_base, f'{split}_labels.csv'), index=False)


# Run From Here

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import kagglehub
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.applications import EfficientNetB3, VGG19, InceptionV3,VGG16
from tensorflow.keras.losses import BinaryFocalCrossentropy
from sklearn.metrics import classification_report, f1_score
import matplotlib.pyplot as plt

In [3]:
rdd_2022_path = kagglehub.dataset_download('aliabdelmenam/rdd-2022')
new_base = rdd_2022_path
print(new_base)
batch_size = 32
target_size = (224, 224)
class_names = ['longitudinal_crack', 'transverse_crack', 'alligator_crack', 'pothole', 'other_damage']
num_classes = len(class_names)
train_df = pd.read_csv("train_labels.csv")
val_df = pd.read_csv("val_labels.csv")
test_df = pd.read_csv("test_labels.csv")

/root/.cache/kagglehub/datasets/aliabdelmenam/rdd-2022/versions/1


In [4]:
# for root, dirs, files in os.walk(new_base):
#     print(f"Directory: {root}")

Directory: /root/.cache/kagglehub/datasets/aliabdelmenam/rdd-2022/versions/1
Directory: /root/.cache/kagglehub/datasets/aliabdelmenam/rdd-2022/versions/1/RDD_SPLIT
Directory: /root/.cache/kagglehub/datasets/aliabdelmenam/rdd-2022/versions/1/RDD_SPLIT/val
Directory: /root/.cache/kagglehub/datasets/aliabdelmenam/rdd-2022/versions/1/RDD_SPLIT/val/images
Directory: /root/.cache/kagglehub/datasets/aliabdelmenam/rdd-2022/versions/1/RDD_SPLIT/val/labels
Directory: /root/.cache/kagglehub/datasets/aliabdelmenam/rdd-2022/versions/1/RDD_SPLIT/test
Directory: /root/.cache/kagglehub/datasets/aliabdelmenam/rdd-2022/versions/1/RDD_SPLIT/test/images
Directory: /root/.cache/kagglehub/datasets/aliabdelmenam/rdd-2022/versions/1/RDD_SPLIT/test/labels
Directory: /root/.cache/kagglehub/datasets/aliabdelmenam/rdd-2022/versions/1/RDD_SPLIT/train
Directory: /root/.cache/kagglehub/datasets/aliabdelmenam/rdd-2022/versions/1/RDD_SPLIT/train/images
Directory: /root/.cache/kagglehub/datasets/aliabdelmenam/rdd-2022/

In [5]:
def create_dataset(df, directory, batch_size, target_size, class_names, shuffle=False):
    def generator():
        indices = df.index.tolist()
        if shuffle:
            np.random.shuffle(indices)
        for i in indices:
            row = df.loc[i]
            img_path = os.path.join(directory, row['filename'])
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=target_size)
            img_array = tf.keras.preprocessing.image.img_to_array(img)
            label = row[class_names].values.astype('float32')
            yield img_array, label

    dataset = tf.data.Dataset.from_generator(
        generator,
        output_signature=(
            tf.TensorSpec(shape=(*target_size, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(num_classes,), dtype=tf.float32),
        )
    )
    if shuffle:
        dataset = dataset.shuffle(1024)
    return dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)


In [7]:
train_dir = os.path.join(new_base, 'RDD_SPLIT/train/images')
val_dir = os.path.join(new_base, 'RDD_SPLIT/val/images')
test_dir = os.path.join(new_base, 'RDD_SPLIT/test/images')



train_ds = create_dataset(train_df, train_dir, batch_size, target_size, class_names, shuffle=True)
val_ds = create_dataset(val_df, val_dir, batch_size, target_size, class_names)
test_ds = create_dataset(test_df, test_dir, batch_size, target_size, class_names)

steps_per_epoch = len(train_df) // batch_size
validation_steps = len(val_df) // batch_size
test_steps = len(test_df) // batch_size

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.25),
    layers.RandomZoom(0.5),
    layers.RandomContrast(0.21),
])

#Modeling Phase

## VGG19

In [ ]:
def build_modelVGG19(input_shape=(224, 224, 3), num_classes=5):
    base_model = VGG19(include_top=False, weights='imagenet', input_shape=input_shape)
    base_model.trainable = False

    inputs = layers.Input(shape=input_shape)
    x = data_augmentation(inputs)
    x = layers.Rescaling(1./255)(x)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='sigmoid')(x)

    model = models.Model(inputs, outputs)

    return model,base_model

In [ ]:
modelVGG19, base_modelVGG19 = build_modelVGG19()
modelVGG19.compile(optimizer='adam', loss=BinaryFocalCrossentropy(gamma=2.0), metrics=['binary_accuracy'])

checkpoint_cb = callbacks.ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_binary_accuracy', mode='max')
early_stop_cb = callbacks.EarlyStopping(patience=6, restore_best_weights=True)

modelVGG19.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[checkpoint_cb, early_stop_cb]
)



80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/20
839/839 ━━━━━━━━━━━━━━━━━━━━ 539s 612ms/step - binary_accuracy: 0.7536 - loss: 0.1435 - val_binary_accuracy: 0.8007 - val_loss: 0.1183
Epoch 2/20
  1/839 ━━━━━━━━━━━━━━━━━━━━ 31:36 2s/step - binary_accuracy: 0.7714 - loss: 0.1328

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


839/839 ━━━━━━━━━━━━━━━━━━━━ 80s 92ms/step - binary_accuracy: 0.7714 - loss: 0.1328 - val_binary_accuracy: 0.8001 - val_loss: 0.1182
Epoch 3/20
839/839 ━━━━━━━━━━━━━━━━━━━━ 568s 654ms/step - binary_accuracy: 0.7917 - loss: 0.1220 - val_binary_accuracy: 0.8027 - val_loss: 0.1168
Epoch 4/20
  1/839 ━━━━━━━━━━━━━━━━━━━━ 1:38 118ms/step - binary_accuracy: 0.8095 - loss: 0.1139

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


839/839 ━━━━━━━━━━━━━━━━━━━━ 77s 92ms/step - binary_accuracy: 0.8095 - loss: 0.1139 - val_binary_accuracy: 0.8019 - val_loss: 0.1167
Epoch 5/20
839/839 ━━━━━━━━━━━━━━━━━━━━ 535s 572ms/step - binary_accuracy: 0.7963 - loss: 0.1198 - val_binary_accuracy: 0.8084 - val_loss: 0.1172
Epoch 6/20
  1/839 ━━━━━━━━━━━━━━━━━━━━ 1:40 120ms/step - binary_accuracy: 0.8000 - loss: 0.1244

...

839/839 ━━━━━━━━━━━━━━━━━━━━ 138s 165ms/step - binary_accuracy: 0.7905 - loss: 0.1194 - val_binary_accuracy: 0.8079 - val_loss: 0.1148
Epoch 13/20
839/839 ━━━━━━━━━━━━━━━━━━━━ 553s 642ms/step - binary_accuracy: 0.7966 - loss: 0.1188 - val_binary_accuracy: 0.8083 - val_loss: 0.1157
Epoch 14/20
  1/839 ━━━━━━━━━━━━━━━━━━━━ 1:53 136ms/step - binary_accuracy: 0.7714 - loss: 0.1245

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


839/839 ━━━━━━━━━━━━━━━━━━━━ 79s 94ms/step - binary_accuracy: 0.7714 - loss: 0.1245 - val_binary_accuracy: 0.8084 - val_loss: 0.1157
Epoch 15/20
839/839 ━━━━━━━━━━━━━━━━━━━━ 546s 575ms/step - binary_accuracy: 0.7957 - loss: 0.1187 - val_binary_accuracy: 0.8038 - val_loss: 0.1152
Epoch 16/20
  1/839 ━━━━━━━━━━━━━━━━━━━━ 1:51 133ms/step - binary_accuracy: 0.8095 - loss: 0.1178

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


839/839 ━━━━━━━━━━━━━━━━━━━━ 83s 99ms/step - binary_accuracy: 0.8095 - loss: 0.1178 - val_binary_accuracy: 0.8038 - val_loss: 0.1152
Epoch 17/20
839/839 ━━━━━━━━━━━━━━━━━━━━ 558s 648ms/step - binary_accuracy: 0.7992 - loss: 0.1177 - val_binary_accuracy: 0.8088 - val_loss: 0.1148


In [ ]:
modelVGG19.load_weights('best_model.keras')
test_loss, test_acc = modelVGG19.evaluate(test_ds.take(test_steps))
print(f"Test Accuracy: {test_acc:.4f} | Loss: {test_loss:.4f}")

y_true = test_df[class_names].values
y_pred_probs = modelVGG19.predict(test_ds.take(test_steps))
y_pred = (y_pred_probs > 0.5).astype(int)

print("\nClassification Report:\n", classification_report(y_true[:len(y_pred)], y_pred, target_names=class_names))
print("Macro F1 Score:", f1_score(y_true[:len(y_pred)], y_pred, average='macro'))

179/179 ━━━━━━━━━━━━━━━━━━━━ 92s 511ms/step - binary_accuracy: 0.8096 - loss: 0.1138
Test Accuracy: 0.8094 | Loss: 0.1137


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


179/179 ━━━━━━━━━━━━━━━━━━━━ 88s 488ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Classification Report:
                     precision    recall  f1-score   support

longitudinal_crack       0.67      0.35      0.46      2066
  transverse_crack       0.40      0.01      0.01      1109
   alligator_crack       0.63      0.03      0.06      1209
           pothole       0.63      0.38      0.47      1094
      other_damage       0.00      0.00      0.00       526

         micro avg       0.65      0.20      0.30      6004
         macro avg       0.47      0.15      0.20      6004
      weighted avg       0.54      0.20      0.26      6004
       samples avg       0.20      0.15      0.16      6004

Macro F1 Score: 0.202129722249106


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

### Fine tuning

In [ ]:
for layer in base_modelVGG19.layers[-5:]:
    layer.trainable = True

modelVGG19.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss = BinaryFocalCrossentropy(gamma=2.0), metrics=['binary_accuracy'])

modelVGG19.fit(
    train_ds,
    validation_data = val_ds,
    epochs=20,
    steps_per_epoch = steps_per_epoch,
    validation_steps = validation_steps,
    callbacks=[checkpoint_cb]
)

In [ ]:
modelVGG19.load_weights('best_model.keras')
test_loss, test_acc = modelVGG19.evaluate(test_ds.take(test_steps))
print(f"Test Accuracy: {test_acc:.4f} | Loss: {test_loss:.4f}")

y_true = test_df[class_names].values
y_pred_probs = modelVGG19.predict(test_ds.take(test_steps))
y_pred = (y_pred_probs > 0.5).astype(int)

print("\nClassification Report:\n", classification_report(y_true[:len(y_pred)], y_pred, target_names=class_names))
print("Macro F1 Score:", f1_score(y_true[:len(y_pred)], y_pred, average='macro'))

## VGG16


In [ ]:
def build_modelVGG16(input_shape=(224, 224, 3), num_classes=5):
    base_model = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
    base_model.trainable = False

    inputs = layers.Input(shape=input_shape)
    x = data_augmentation(inputs)
    x = layers.Rescaling(1./255)(x)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.35)(x)
    outputs = layers.Dense(num_classes, activation='sigmoid')(x)

    model = models.Model(inputs, outputs)

    return model,base_model

In [ ]:
modelVGG16, base_modelVGG16 = build_modelVGG16()
modelVGG16.compile(optimizer='adam', loss=BinaryFocalCrossentropy(gamma=2.5), metrics=['binary_accuracy'])

checkpoint_cb = callbacks.ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_binary_accuracy', mode='max')
early_stop_cb = callbacks.EarlyStopping(patience=6, restore_best_weights=True)

modelVGG16.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    # steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[checkpoint_cb, early_stop_cb]
)



58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/20
    840/Unknown 423s 483ms/step - binary_accuracy: 0.7484 - loss: 0.1046

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


840/840 ━━━━━━━━━━━━━━━━━━━━ 512s 589ms/step - binary_accuracy: 0.7485 - loss: 0.1046 - val_binary_accuracy: 0.7973 - val_loss: 0.0841
Epoch 2/20
840/840 ━━━━━━━━━━━━━━━━━━━━ 0s 470ms/step - binary_accuracy: 0.7951 - loss: 0.0855

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


840/840 ━━━━━━━━━━━━━━━━━━━━ 552s 576ms/step - binary_accuracy: 0.7951 - loss: 0.0855 - val_binary_accuracy: 0.8060 - val_loss: 0.0827
Epoch 3/20
840/840 ━━━━━━━━━━━━━━━━━━━━ 0s 460ms/step - binary_accuracy: 0.7973 - loss: 0.0845

...

840/840 ━━━━━━━━━━━━━━━━━━━━ 488s 565ms/step - binary_accuracy: 0.7995 - loss: 0.0832 - val_binary_accuracy: 0.8075 - val_loss: 0.0811
Epoch 14/20
840/840 ━━━━━━━━━━━━━━━━━━━━ 0s 455ms/step - binary_accuracy: 0.7983 - loss: 0.0835

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


840/840 ━━━━━━━━━━━━━━━━━━━━ 484s 560ms/step - binary_accuracy: 0.7983 - loss: 0.0835 - val_binary_accuracy: 0.8111 - val_loss: 0.0810
Epoch 15/20
840/840 ━━━━━━━━━━━━━━━━━━━━ 0s 459ms/step - binary_accuracy: 0.7974 - loss: 0.0838

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


840/840 ━━━━━━━━━━━━━━━━━━━━ 504s 563ms/step - binary_accuracy: 0.7974 - loss: 0.0838 - val_binary_accuracy: 0.8103 - val_loss: 0.0816
Epoch 16/20
145/840 ━━━━━━━━━━━━━━━━━━━━ 5:21 462ms/step - binary_accuracy: 0.7978 - loss: 0.0841

In [ ]:
modelVGG16.load_weights('best_model.keras')
test_loss, test_acc = modelVGG16.evaluate(test_ds.take(test_steps))
print(f"Test Accuracy: {test_acc:.4f} | Loss: {test_loss:.4f}")

y_true = test_df[class_names].values
y_pred_probs = modelVGG16.predict(test_ds.take(test_steps))
y_pred = (y_pred_probs > 0.5).astype(int)

print("\nClassification Report:\n", classification_report(y_true[:len(y_pred)], y_pred, target_names=class_names))
print("Macro F1 Score:", f1_score(y_true[:len(y_pred)], y_pred, average='macro'))

### Fine tuning

In [ ]:
for layer in base_modelVGG16.layers[-5:]:
    layer.trainable = True

modelVGG16.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss = BinaryFocalCrossentropy(gamma=2.0), metrics=['binary_accuracy'])

modelVGG16.fit(
    train_ds,
    validation_data = val_ds,
    epochs=10,
    steps_per_epoch = steps_per_epoch,
    validation_steps = validation_steps,
    callbacks=[checkpoint_cb]
)

In [ ]:
modelVGG16.load_weights('best_model.keras')
test_loss, test_acc = modelVGG16.evaluate(test_ds.take(test_steps))
print(f"Test Accuracy: {test_acc:.4f} | Loss: {test_loss:.4f}")

y_true = test_df[class_names].values
y_pred_probs = modelVGG16.predict(test_ds.take(test_steps))
y_pred = (y_pred_probs > 0.5).astype(int)

print("\nClassification Report:\n", classification_report(y_true[:len(y_pred)], y_pred, target_names=class_names))
print("Macro F1 Score:", f1_score(y_true[:len(y_pred)], y_pred, average='macro'))